In [1]:
#------------------------
# TODO address all TODOs in dovs_and_summary_usage_helper_functions.ipynb
%run dovs_and_summary_usage_helper_functions.ipynb

In [2]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import itertools
import copy
import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_Box_sns
import GrubbsTest

# -------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------

# Example 1

### Select: Super simple

In [3]:
cols_of_interest = ['CI_NB', 'CMI_NB', 'OUTG_REC_NB', 'OUTAGE_NB', 'DT_ON_TS', 'DT_OFF_TS']
sql_select = SQLSelect(cols_of_interest)
sql_select.print()

SELECT
	CI_NB,
	CMI_NB,
	OUTG_REC_NB,
	OUTAGE_NB,
	DT_ON_TS,
	DT_OFF_TS


### Select: Slightly more involved

In [4]:
# ****************************************************************
# SELECT STATEMENT
sql_select = SQLSelect(field_descs=['CI_NB', 'CMI_NB', 'OUTG_REC_NB', 'OUTAGE_NB', 'DT_ON_TS', 'DT_OFF_TS', 
                                      dict(field_desc='DOV.DT_ON_TS - DOV.STEP_DRTN_NB/(60*24)', alias='DT_OFF_TS_FULL', table_alias_prefix=None), 
                                      'STEP_DRTN_NB', 
                                      dict(field_desc='EXTRACT(YEAR FROM DOV.DT_OFF_TS)', alias='START_YEAR', table_alias_prefix=None), 
                                      'OPERATING_UNIT_ID', 'STATE_ABBR_TX', 'MJR_CAUSE_CD', 'MNR_CAUSE_CD', 'LOCATION_ID', 'GIS_CRCT_NB'], 
                         global_table_alias_prefix='DOV', idxs=None, run_check=True)

sql_select.add_select_element(field_desc='OPRTG_UNT_NM', alias=None, 
                                table_alias_prefix='DOV1', idx=None, run_check=True)
sql_select.add_select_elements(field_descs=['MJR_CAUSE_NM', 'MNR_CAUSE_NM'], 
                                 global_table_alias_prefix='DOV5', idxs=None, run_check=True)
sql_select.add_select_elements(field_descs=['OFF_TM', 'REST_TM', 'PREMISE_NB'],
                                 global_table_alias_prefix='PRIM', idxs=None, run_check=True)

In [5]:
sql_select.print()

SELECT
	DOV.CI_NB,
	DOV.CMI_NB,
	DOV.OUTG_REC_NB,
	DOV.OUTAGE_NB,
	DOV.DT_ON_TS,
	DOV.DT_OFF_TS,
	DOV.DT_ON_TS - DOV.STEP_DRTN_NB/(60*24) AS DT_OFF_TS_FULL,
	DOV.STEP_DRTN_NB,
	EXTRACT(YEAR FROM DOV.DT_OFF_TS) AS START_YEAR,
	DOV.OPERATING_UNIT_ID,
	DOV.STATE_ABBR_TX,
	DOV.MJR_CAUSE_CD,
	DOV.MNR_CAUSE_CD,
	DOV.LOCATION_ID,
	DOV.GIS_CRCT_NB,
	DOV1.OPRTG_UNT_NM,
	DOV5.MJR_CAUSE_NM,
	DOV5.MNR_CAUSE_NM,
	PRIM.OFF_TM,
	PRIM.REST_TM,
	PRIM.PREMISE_NB


In [6]:
sql_select.print(include_alias=False)

SELECT
	DOV.CI_NB,
	DOV.CMI_NB,
	DOV.OUTG_REC_NB,
	DOV.OUTAGE_NB,
	DOV.DT_ON_TS,
	DOV.DT_OFF_TS,
	DOV.DT_ON_TS - DOV.STEP_DRTN_NB/(60*24),
	DOV.STEP_DRTN_NB,
	EXTRACT(YEAR FROM DOV.DT_OFF_TS),
	DOV.OPERATING_UNIT_ID,
	DOV.STATE_ABBR_TX,
	DOV.MJR_CAUSE_CD,
	DOV.MNR_CAUSE_CD,
	DOV.LOCATION_ID,
	DOV.GIS_CRCT_NB,
	DOV1.OPRTG_UNT_NM,
	DOV5.MJR_CAUSE_NM,
	DOV5.MNR_CAUSE_NM,
	PRIM.OFF_TM,
	PRIM.REST_TM,
	PRIM.PREMISE_NB


In [7]:
sql_select.print(include_alias=False, include_table_alias_prefix=False)

SELECT
	CI_NB,
	CMI_NB,
	OUTG_REC_NB,
	OUTAGE_NB,
	DT_ON_TS,
	DT_OFF_TS,
	DOV.DT_ON_TS - DOV.STEP_DRTN_NB/(60*24),
	STEP_DRTN_NB,
	EXTRACT(YEAR FROM DOV.DT_OFF_TS),
	OPERATING_UNIT_ID,
	STATE_ABBR_TX,
	MJR_CAUSE_CD,
	MNR_CAUSE_CD,
	LOCATION_ID,
	GIS_CRCT_NB,
	OPRTG_UNT_NM,
	MJR_CAUSE_NM,
	MNR_CAUSE_NM,
	OFF_TM,
	REST_TM,
	PREMISE_NB


# From and Join

In [8]:
# ****************************************************************
# FROM AND JOIN STATEMENTS
sql_from = SQLFrom(schema_name='DOVSADM', table_name='DOVS_OUTAGE_FACT', alias='DOV')
sql_from.print()

FROM DOVSADM.DOVS_OUTAGE_FACT DOV


In [9]:
join_type = 'LEFT OUTER'
join_table = 'DOVSADM.DOVS_MASTER_GEO_DIM'
join_table_alias = 'DOV1'
orig_table_alias = 'DOV'
list_of_columns_to_join = [
    ['OPERATING_UNIT_ID', 'OPRTG_UNT_ID'], 
    ['STATE_ABBR_TX', 'STATE_ID'], 
    ['OPCO_NBR', 'OPCO_ID'], 
    ['DISTRICT_NB', 'DISTRICT_ID'], 
    ['SRVC_CNTR_NB', 'AREA_ID'], 
    ['GIS_CRCT_NB', 'GIS_CIRCUIT_ID']
]
sql_join = SQLJoin(join_type=join_type, join_table=join_table, 
                   join_table_alias=join_table_alias, orig_table_alias=orig_table_alias, 
                   list_of_columns_to_join=list_of_columns_to_join)
sql_from.add_join_to_coll(sql_join, idx=None, run_check=True)
sql_from.print()

FROM DOVSADM.DOVS_OUTAGE_FACT DOV
	LEFT OUTER JOIN DOVSADM.DOVS_MASTER_GEO_DIM DOV1 ON DOV.OPERATING_UNIT_ID=DOV1.OPRTG_UNT_ID AND DOV.STATE_ABBR_TX=DOV1.STATE_ID AND DOV.OPCO_NBR=DOV1.OPCO_ID AND DOV.DISTRICT_NB=DOV1.DISTRICT_ID AND DOV.SRVC_CNTR_NB=DOV1.AREA_ID AND DOV.GIS_CRCT_NB=DOV1.GIS_CIRCUIT_ID


In [10]:
sql_from.build_and_add_join_to_coll(join_type = 'LEFT OUTER', 
                                    join_table = 'DOVSADM.DOVS_OUTAGE_ATTRIBUTES_DIM', 
                                    join_table_alias = 'DOV2', 
                                    orig_table_alias = 'DOV', 
                                    list_of_columns_to_join = ['OUTG_REC_NB'], 
                                    idx=None, run_check=True)
sql_from.print()

FROM DOVSADM.DOVS_OUTAGE_FACT DOV
	LEFT OUTER JOIN DOVSADM.DOVS_MASTER_GEO_DIM DOV1 ON DOV.OPERATING_UNIT_ID=DOV1.OPRTG_UNT_ID AND DOV.STATE_ABBR_TX=DOV1.STATE_ID AND DOV.OPCO_NBR=DOV1.OPCO_ID AND DOV.DISTRICT_NB=DOV1.DISTRICT_ID AND DOV.SRVC_CNTR_NB=DOV1.AREA_ID AND DOV.GIS_CRCT_NB=DOV1.GIS_CIRCUIT_ID
	LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_ATTRIBUTES_DIM DOV2 ON DOV.OUTG_REC_NB=DOV2.OUTG_REC_NB


In [11]:
sql_from = DOVSOutages_SQL.add_std_join_DOVS_OUTAGE_FACT_w_DOVS_OUTAGE_CAUSE_TYPES_DIM(add_to_me=sql_from, alias_DOVS_OUTAGE_FACT='DOV', alias_DOVS_OUTAGE_CAUSE_TYPES_DIM='DOV5')
sql_from = DOVSOutages_SQL.add_std_join_DOVS_OUTAGE_FACT_w_DOVS_PREMISE_DIM(add_to_me=sql_from, alias_DOVS_OUTAGE_FACT='DOV', alias_DOVS_PREMISE_DIM='PRIM')
sql_from.print()

FROM DOVSADM.DOVS_OUTAGE_FACT DOV
	LEFT OUTER JOIN DOVSADM.DOVS_MASTER_GEO_DIM DOV1 ON DOV.OPERATING_UNIT_ID=DOV1.OPRTG_UNT_ID AND DOV.STATE_ABBR_TX=DOV1.STATE_ID AND DOV.OPCO_NBR=DOV1.OPCO_ID AND DOV.DISTRICT_NB=DOV1.DISTRICT_ID AND DOV.SRVC_CNTR_NB=DOV1.AREA_ID AND DOV.GIS_CRCT_NB=DOV1.GIS_CIRCUIT_ID
	LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_ATTRIBUTES_DIM DOV2 ON DOV.OUTG_REC_NB=DOV2.OUTG_REC_NB
	LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_CAUSE_TYPES_DIM DOV5 ON DOV.MJR_CAUSE_CD=DOV5.MJR_CAUSE_CD AND DOV.MNR_CAUSE_CD=DOV5.MNR_CAUSE_CD
	LEFT OUTER JOIN DOVSADM.DOVS_PREMISE_DIM PRIM ON DOV.OUTG_REC_NB=PRIM.OUTG_REC_NB


# Where

In [12]:
premise_nbs = [1234, 2345, 3456, 4567, 5678]

In [13]:
sql_where = SQLWhere([dict(field_desc='PREMISE_NB', table_alias_prefix='PRIM', comparison_operator='=', value='{}', needs_quotes=True), 
                      dict(field_desc='INTRPTN_TYP_CD', table_alias_prefix='DOV2', comparison_operator='=', value='S', needs_quotes=True), 
                      dict(field_desc='CURR_REC_STAT_CD', table_alias_prefix='DOV2', comparison_operator='=', value='A', needs_quotes=True), 
                      dict(field_desc='MJR_CAUSE_CD', table_alias_prefix='DOV', comparison_operator='<>', value='NI', needs_quotes=True), 
                      dict(field_desc='DEVICE_CD', table_alias_prefix='DOV', comparison_operator='<>', value=85, needs_quotes=False), 
                      dict(field_desc='STATE_ABBR_TX', table_alias_prefix='DOV', comparison_operator='=', value='OH', needs_quotes=True)], 
                     idxs=None, run_check=True)
sql_where.print()

WHERE PRIM.PREMISE_NB = '{}'
AND   DOV2.INTRPTN_TYP_CD = 'S'
AND   DOV2.CURR_REC_STAT_CD = 'A'
AND   DOV.MJR_CAUSE_CD <> 'NI'
AND   DOV.DEVICE_CD <> 85
AND   DOV.STATE_ABBR_TX = 'OH'


In [14]:
sql_where = SQLWhere([dict(field_desc='PREMISE_NB', table_alias_prefix='PRIM', comparison_operator='=', 
                           value=f'({Utilities_sql.join_list_w_quotes(premise_nbs)})', needs_quotes=False), 
                      dict(field_desc='INTRPTN_TYP_CD', table_alias_prefix='DOV2', comparison_operator='=', value='S', needs_quotes=True), 
                      dict(field_desc='CURR_REC_STAT_CD', table_alias_prefix='DOV2', comparison_operator='=', value='A', needs_quotes=True), 
                      dict(field_desc='MJR_CAUSE_CD', table_alias_prefix='DOV', comparison_operator='<>', value='NI', needs_quotes=True), 
                      dict(field_desc='DEVICE_CD', table_alias_prefix='DOV', comparison_operator='<>', value=85, needs_quotes=False), 
                      dict(field_desc='STATE_ABBR_TX', table_alias_prefix='DOV', comparison_operator='=', value='OH', needs_quotes=True)], 
                     idxs=None, run_check=True)
sql_where.print()

WHERE PRIM.PREMISE_NB = ('1234','2345','3456','4567','5678')
AND   DOV2.INTRPTN_TYP_CD = 'S'
AND   DOV2.CURR_REC_STAT_CD = 'A'
AND   DOV.MJR_CAUSE_CD <> 'NI'
AND   DOV.DEVICE_CD <> 85
AND   DOV.STATE_ABBR_TX = 'OH'


# SQLQuery

In [15]:
sql_query = SQLQuery(sql_select=sql_select, 
                     sql_from=sql_from, 
                     sql_where=sql_where)
sql_query.print()

SELECT
	DOV.CI_NB,
	DOV.CMI_NB,
	DOV.OUTG_REC_NB,
	DOV.OUTAGE_NB,
	DOV.DT_ON_TS,
	DOV.DT_OFF_TS,
	DOV.DT_ON_TS - DOV.STEP_DRTN_NB/(60*24) AS DT_OFF_TS_FULL,
	DOV.STEP_DRTN_NB,
	EXTRACT(YEAR FROM DOV.DT_OFF_TS) AS START_YEAR,
	DOV.OPERATING_UNIT_ID,
	DOV.STATE_ABBR_TX,
	DOV.MJR_CAUSE_CD,
	DOV.MNR_CAUSE_CD,
	DOV.LOCATION_ID,
	DOV.GIS_CRCT_NB,
	DOV1.OPRTG_UNT_NM,
	DOV5.MJR_CAUSE_NM,
	DOV5.MNR_CAUSE_NM,
	PRIM.OFF_TM,
	PRIM.REST_TM,
	PRIM.PREMISE_NB
FROM DOVSADM.DOVS_OUTAGE_FACT DOV
	LEFT OUTER JOIN DOVSADM.DOVS_MASTER_GEO_DIM DOV1 ON DOV.OPERATING_UNIT_ID=DOV1.OPRTG_UNT_ID AND DOV.STATE_ABBR_TX=DOV1.STATE_ID AND DOV.OPCO_NBR=DOV1.OPCO_ID AND DOV.DISTRICT_NB=DOV1.DISTRICT_ID AND DOV.SRVC_CNTR_NB=DOV1.AREA_ID AND DOV.GIS_CRCT_NB=DOV1.GIS_CIRCUIT_ID
	LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_ATTRIBUTES_DIM DOV2 ON DOV.OUTG_REC_NB=DOV2.OUTG_REC_NB
	LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_CAUSE_TYPES_DIM DOV5 ON DOV.MJR_CAUSE_CD=DOV5.MJR_CAUSE_CD AND DOV.MNR_CAUSE_CD=DOV5.MNR_CAUSE_CD
	LEFT OUTER JOIN DO

# -------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------

# Example 2: With GroupBy and aggregate metrics

In [16]:
cols_of_interest_usage_agg = ['starttimeperiod', 'endtimeperiod', 'aep_endtime_utc', 'timezoneoffset', 
                              'aep_derived_uom', 'aep_srvc_qlty_idntfr', 'value', 'aep_usage_dt']

serial_numbers = [884390632, 880682782, 880320285, 880076534, 880320207, 889926007, 884390655, 880076535]
date_range = ['2021-01-01', '2021-01-02']
agg_cols_and_types = {'value':['sum', 'sq_sum', 'mean', 'std', 'count']}
groupby_cols = ['starttimeperiod', 'endtimeperiod', 'aep_endtime_utc', 'timezoneoffset', 
                'aep_derived_uom', 'aep_srvc_qlty_idntfr', 'aep_usage_dt']

In [17]:
sql_select = SQLSelect(cols_of_interest_usage_agg)
sql_select.print()

SELECT
	starttimeperiod,
	endtimeperiod,
	aep_endtime_utc,
	timezoneoffset,
	aep_derived_uom,
	aep_srvc_qlty_idntfr,
	value,
	aep_usage_dt


In [18]:
sql_from = SQLFrom(schema_name='usage_nonvee', table_name='reading_ivl_nonvee', alias=None)
sql_from.print()

FROM usage_nonvee.reading_ivl_nonvee


In [19]:
sql_where = SQLWhere([dict(field_desc='aep_opco', comparison_operator='=', value='oh', needs_quotes=True), 
                      dict(field_desc='aep_usage_dt', comparison_operator='BETWEEN', 
                           value=[f'{date_range[0]}', f'{date_range[1]}'], needs_quotes=True), 
                      dict(field_desc='serialnumber', comparison_operator='IN', 
                           value=f'({Utilities_sql.join_list_w_quotes(serial_numbers)})', needs_quotes=False)], 
                     idxs=None, run_check=True)
sql_where.print()

WHERE aep_opco = 'oh'
AND   aep_usage_dt BETWEEN '2021-01-01' AND '2021-01-02'
AND   serialnumber IN ('884390632','880682782','880320285','880076534','880320207','889926007','884390655','880076535')


In [20]:
sql_select.print()

SELECT
	starttimeperiod,
	endtimeperiod,
	aep_endtime_utc,
	timezoneoffset,
	aep_derived_uom,
	aep_srvc_qlty_idntfr,
	value,
	aep_usage_dt


In [21]:
sql_select.add_aggregate_elements(agg_cols_and_types=agg_cols_and_types, 
                                  include_counts_including_null=True)
sql_select.print()

SELECT
	starttimeperiod,
	endtimeperiod,
	aep_endtime_utc,
	timezoneoffset,
	aep_derived_uom,
	aep_srvc_qlty_idntfr,
	aep_usage_dt,
	SUM(value) AS sum_value,
	SUM(POWER(value, 2)) AS sq_sum_value,
	AVG(value) AS mean_value,
	STDDEV_SAMP(value) AS std_value,
	COUNT(value) AS count_value,
	COUNT(*) AS counts_including_null


### Note: add_aggregate_elements removed the 'normal' value field

In [22]:
sql_groupby = SQLGroupBy(field_descs=groupby_cols, 
                         global_table_alias_prefix=None, 
                         idxs=None, run_check=True)
sql_groupby.print()

GROUP BY
	starttimeperiod,
	endtimeperiod,
	aep_endtime_utc,
	timezoneoffset,
	aep_derived_uom,
	aep_srvc_qlty_idntfr,
	aep_usage_dt


In [23]:
sql_query = SQLQuery(sql_select = sql_select, 
                     sql_from = sql_from, 
                     sql_where = sql_where, 
                     sql_groupby = sql_groupby
                    )
sql_query.print()

SELECT
	starttimeperiod,
	endtimeperiod,
	aep_endtime_utc,
	timezoneoffset,
	aep_derived_uom,
	aep_srvc_qlty_idntfr,
	aep_usage_dt,
	SUM(value) AS sum_value,
	SUM(POWER(value, 2)) AS sq_sum_value,
	AVG(value) AS mean_value,
	STDDEV_SAMP(value) AS std_value,
	COUNT(value) AS count_value,
	COUNT(*) AS counts_including_null
FROM usage_nonvee.reading_ivl_nonvee
WHERE aep_opco = 'oh'
AND   aep_usage_dt BETWEEN '2021-01-01' AND '2021-01-02'
AND   serialnumber IN ('884390632','880682782','880320285','880076534','880320207','889926007','884390655','880076535')
GROUP BY
	starttimeperiod,
	endtimeperiod,
	aep_endtime_utc,
	timezoneoffset,
	aep_derived_uom,
	aep_srvc_qlty_idntfr,
	aep_usage_dt


# -------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------

# At this point, you're probably thinking...
'Cool, but this doesn't seem any easier than simply writing out the SQL statement by hand."
<br> You're exactly right!  In fact, at this level of implementation, this is possibly more difficult than writing out by hand.
<br> HOWEVER, the point is that this can be used as a base from which to build more powerful tools!
- these simple building blocks can construct complex structures

# -------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------

# Example 3: Using AMINonVee_SQL.build_sql_usg

In [24]:
cols_of_interest_usage_agg = ['starttimeperiod', 'endtimeperiod', 'aep_endtime_utc', 'timezoneoffset', 
                              'aep_derived_uom', 'aep_srvc_qlty_idntfr', 'value', 'aep_usage_dt']

serial_numbers = [884390632, 880682782, 880320285, 880076534, 880320207, 889926007, 884390655, 880076535]
date_range = ['2021-01-01', '2021-01-02']
agg_cols_and_types = {'value':['sum', 'sq_sum', 'mean', 'std', 'count']}
groupby_cols = ['starttimeperiod', 'endtimeperiod', 'aep_endtime_utc', 'timezoneoffset', 
                'aep_derived_uom', 'aep_srvc_qlty_idntfr', 'aep_usage_dt']

In [25]:
sql_query = AMINonVee_SQL.build_sql_usg(
    cols_of_interest=cols_of_interest_usage_agg, 
    aep_derived_uoms_and_idntfrs=None, 
    kwh_and_vlt_only=False, 
    aep_opco='oh', 
    serial_numbers=serial_numbers, 
    date_range=date_range, 
    agg_cols_and_types=agg_cols_and_types, 
    groupby_cols=groupby_cols
)
sql_query.print()

SELECT
	starttimeperiod,
	endtimeperiod,
	aep_endtime_utc,
	timezoneoffset,
	aep_derived_uom,
	aep_srvc_qlty_idntfr,
	aep_usage_dt,
	SUM(value) AS sum_value,
	SUM(POWER(value, 2)) AS sq_sum_value,
	AVG(value) AS mean_value,
	STDDEV_SAMP(value) AS std_value,
	COUNT(value) AS count_value,
	COUNT(*) AS counts_including_null
FROM usage_nonvee.reading_ivl_nonvee
WHERE aep_opco = 'oh'
AND   aep_usage_dt BETWEEN '2021-01-01' AND '2021-01-02'
AND   serialnumber IN ('884390632','880682782','880320285','880076534','880320207','889926007','884390655','880076535')
GROUP BY
	starttimeperiod,
	endtimeperiod,
	aep_endtime_utc,
	timezoneoffset,
	aep_derived_uom,
	aep_srvc_qlty_idntfr,
	aep_usage_dt


# -------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------

# Other Examples

In [26]:
cols_of_interest_usage = ['serialnumber', 'starttimeperiod', 'endtimeperiod', 'aep_endtime_utc', 'timezoneoffset',
                          'aep_derived_uom', 'aep_srvc_qlty_idntfr', 'value', 'aep_usage_dt']
serial_numbers = [884390632, 880682782, 880320285, 880076534, 880320207, 889926007, 884390655, 880076535]
date_range = ['2021-01-01', '2021-01-02']
agg_cols_and_types = {'value':['sum', 'sq_sum', 'mean', 'std', 'count']}
groupby_cols = ['starttimeperiod', 'endtimeperiod', 'aep_endtime_utc', 'timezoneoffset', 
                'aep_derived_uom', 'aep_srvc_qlty_idntfr', 'aep_usage_dt']
try_to_split_col_strs = True    
#-------------------------------------------------------------

In [27]:
sql_query = AMINonVee_SQL.build_sql_usg(
    cols_of_interest_usage, 
    aep_derived_uoms_and_idntfrs=None, 
    kwh_and_vlt_only=False, 
    return_statement=True, 
    date_range=date_range, 
    aep_premise_nbs =[1,2,3],
    serial_numbers=serial_numbers
)
sql_query.print()

SELECT
	serialnumber,
	starttimeperiod,
	endtimeperiod,
	aep_endtime_utc,
	timezoneoffset,
	aep_derived_uom,
	aep_srvc_qlty_idntfr,
	value,
	aep_usage_dt
FROM usage_nonvee.reading_ivl_nonvee
WHERE aep_usage_dt BETWEEN '2021-01-01' AND '2021-01-02'
AND   serialnumber IN ('884390632','880682782','880320285','880076534','880320207','889926007','884390655','880076535')
AND   aep_premise_nb IN ('1','2','3')


In [28]:
AMINonVee_SQL.build_sql_usg(
    cols_of_interest_usage, 
    aep_derived_uoms_and_idntfrs=['KWH', ['VOLT', 'AVG'], 'KVARH'], 
    kwh_and_vlt_only=False, 
    return_statement=True, 
    date_range=date_range, 
    serial_numbers=serial_numbers
).print()

SELECT
	serialnumber,
	starttimeperiod,
	endtimeperiod,
	aep_endtime_utc,
	timezoneoffset,
	aep_derived_uom,
	aep_srvc_qlty_idntfr,
	value,
	aep_usage_dt
FROM usage_nonvee.reading_ivl_nonvee
WHERE aep_usage_dt BETWEEN '2021-01-01' AND '2021-01-02'
AND   serialnumber IN ('884390632','880682782','880320285','880076534','880320207','889926007','884390655','880076535')
AND   (
	aep_derived_uom = 'KWH' OR 
	(
		aep_derived_uom = 'VOLT' AND 
		aep_srvc_qlty_idntfr = 'AVG'
	) OR 
	aep_derived_uom = 'KVARH'
)


In [29]:
AMINonVee_SQL.build_sql_usg(
    cols_of_interest_usage, 
    aep_derived_uoms_and_idntfrs=['KWH', ['VOLT', 'AVG'], 'KVARH'], 
    kwh_and_vlt_only=False, 
    return_statement=True, 
    date_range=date_range, 
    serial_numbers=serial_numbers, 
    groupby_cols=groupby_cols, 
    agg_cols_and_types=agg_cols_and_types
).print()

SELECT
	serialnumber,
	starttimeperiod,
	endtimeperiod,
	aep_endtime_utc,
	timezoneoffset,
	aep_derived_uom,
	aep_srvc_qlty_idntfr,
	aep_usage_dt,
	SUM(value) AS sum_value,
	SUM(POWER(value, 2)) AS sq_sum_value,
	AVG(value) AS mean_value,
	STDDEV_SAMP(value) AS std_value,
	COUNT(value) AS count_value,
	COUNT(*) AS counts_including_null
FROM usage_nonvee.reading_ivl_nonvee
WHERE aep_usage_dt BETWEEN '2021-01-01' AND '2021-01-02'
AND   serialnumber IN ('884390632','880682782','880320285','880076534','880320207','889926007','884390655','880076535')
AND   (
	aep_derived_uom = 'KWH' OR 
	(
		aep_derived_uom = 'VOLT' AND 
		aep_srvc_qlty_idntfr = 'AVG'
	) OR 
	aep_derived_uom = 'KVARH'
)
GROUP BY
	starttimeperiod,
	endtimeperiod,
	aep_endtime_utc,
	timezoneoffset,
	aep_derived_uom,
	aep_srvc_qlty_idntfr,
	aep_usage_dt


In [30]:
print(AMINonVee_SQL.build_net_kwh_usage_sql_statement(cols_of_interest_usage, 
                                                      additional_derived_uoms=None, 
                                                      run_careful=True, 
                                                      return_statement=True, 
                                                      date_range=date_range, 
                                                      serial_numbers=serial_numbers))

KWH_USG_DELREC_W_SIGNED_VAL AS (
	SELECT
		USG_i.serialnumber,
		USG_i.starttimeperiod,
		USG_i.endtimeperiod,
		USG_i.aep_endtime_utc,
		USG_i.timezoneoffset,
		USG_i.aep_derived_uom,
		USG_i.aep_srvc_qlty_idntfr,
		USG_i.value,
		USG_i.aep_usage_dt,
		IF(aep_srvc_qlty_idntfr='RECEIVED', -1*value, value) AS signed_value
	FROM usage_nonvee.reading_ivl_nonvee USG_i
	WHERE USG_i.aep_usage_dt BETWEEN '2021-01-01' AND '2021-01-02'
	AND   USG_i.serialnumber IN ('884390632','880682782','880320285','880076534','880320207','889926007','884390655','880076535')
	AND   (
		(
			aep_derived_uom = 'KWH' AND 
			aep_srvc_qlty_idntfr = 'RECEIVED'
		) OR 
		(
			aep_derived_uom = 'KWH' AND 
			aep_srvc_qlty_idntfr = 'DELIVERED'
		)
	)
), 

KWH_USG_DELREC_NET AS (
	SELECT
		serialnumber,
		starttimeperiod,
		endtimeperiod,
		aep_endtime_utc,
		timezoneoffset,
		aep_derived_uom,
		'DEL_MINUS_REC' AS aep_srvc_qlty_idntfr,
		SUM(signed_value) AS value,
		aep_usage_dt
	FROM KWH_USG_DELREC_W_SIGNED_VAL
	GRO

In [31]:
print(AMINonVee_SQL.build_net_kwh_usage_sql_statement(cols_of_interest_usage, 
                                                      additional_derived_uoms='ALL', 
                                                      run_careful=False, 
                                                      return_statement=True, 
                                                      date_range=date_range, 
                                                      serial_numbers=serial_numbers))

KWH_USG_DELREC_W_SIGNED_VAL AS (
	SELECT
		USG_i.serialnumber,
		USG_i.starttimeperiod,
		USG_i.endtimeperiod,
		USG_i.aep_endtime_utc,
		USG_i.timezoneoffset,
		USG_i.aep_derived_uom,
		USG_i.aep_srvc_qlty_idntfr,
		USG_i.value,
		USG_i.aep_usage_dt,
		IF(aep_srvc_qlty_idntfr='RECEIVED', -1*value, value) AS signed_value
	FROM usage_nonvee.reading_ivl_nonvee USG_i
	WHERE USG_i.aep_usage_dt BETWEEN '2021-01-01' AND '2021-01-02'
	AND   USG_i.serialnumber IN ('884390632','880682782','880320285','880076534','880320207','889926007','884390655','880076535')
	AND   (
		(
			aep_derived_uom = 'KWH' AND 
			aep_srvc_qlty_idntfr = 'RECEIVED'
		) OR 
		(
			aep_derived_uom = 'KWH' AND 
			aep_srvc_qlty_idntfr = 'DELIVERED'
		)
	)
), 

KWH_USG_DELREC_NET AS (
	SELECT
		serialnumber,
		starttimeperiod,
		endtimeperiod,
		aep_endtime_utc,
		timezoneoffset,
		aep_derived_uom,
		'DEL_MINUS_REC' AS aep_srvc_qlty_idntfr,
		SUM(signed_value) AS value,
		aep_usage_dt
	FROM KWH_USG_DELREC_W_SIGNED_VAL
	GRO

In [32]:
print(AMINonVee_SQL.build_net_kwh_usage_sql_statement(cols_of_interest_usage, 
                                                      additional_derived_uoms='ALL', 
                                                      run_careful=False, 
                                                      return_statement=True, 
                                                      date_range=date_range, 
                                                      serial_numbers=serial_numbers))

KWH_USG_DELREC_W_SIGNED_VAL AS (
	SELECT
		USG_i.serialnumber,
		USG_i.starttimeperiod,
		USG_i.endtimeperiod,
		USG_i.aep_endtime_utc,
		USG_i.timezoneoffset,
		USG_i.aep_derived_uom,
		USG_i.aep_srvc_qlty_idntfr,
		USG_i.value,
		USG_i.aep_usage_dt,
		IF(aep_srvc_qlty_idntfr='RECEIVED', -1*value, value) AS signed_value
	FROM usage_nonvee.reading_ivl_nonvee USG_i
	WHERE USG_i.aep_usage_dt BETWEEN '2021-01-01' AND '2021-01-02'
	AND   USG_i.serialnumber IN ('884390632','880682782','880320285','880076534','880320207','889926007','884390655','880076535')
	AND   (
		(
			aep_derived_uom = 'KWH' AND 
			aep_srvc_qlty_idntfr = 'RECEIVED'
		) OR 
		(
			aep_derived_uom = 'KWH' AND 
			aep_srvc_qlty_idntfr = 'DELIVERED'
		)
	)
), 

KWH_USG_DELREC_NET AS (
	SELECT
		serialnumber,
		starttimeperiod,
		endtimeperiod,
		aep_endtime_utc,
		timezoneoffset,
		aep_derived_uom,
		'DEL_MINUS_REC' AS aep_srvc_qlty_idntfr,
		SUM(signed_value) AS value,
		aep_usage_dt
	FROM KWH_USG_DELREC_W_SIGNED_VAL
	GRO